In [ ]:
import six
import numpy as np
import tensorflow.compat.v2 as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import keras
!pip install git+https://github.com/google/qkeras
from qkeras.quantizers import quantized_bits

  Cloning https://github.com/google/qkeras to /tmp/pip-req-build-6hv98wef
  Running command git clone --filter=blob:none --quiet https://github.com/google/qkeras /tmp/pip-req-build-6hv98wef
  Resolved https://github.com/google/qkeras to commit 6be2f5ca75d9a42209b17f42e86087f6d60cf961
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for QKeras: filename=QKeras-0.9.0-py3-none-any.whl size=175446 sha256=eec77953fe90fb2768acef0bdc7671450ea1279d18b75ec7c6a9dcb056401eee
  Stored in directory: /tmp/pip-ephem-wheel-cache-wmqn2267/wheels/9e/fd/c9/9e660ece157054c7d9225b4ce0e010e5bfa4812a349c96d247
  Created wheel for pyparser: filename=pyparser-1.0-py3-none-any.whl size=4915 sha256=c1c65a6aa7e9c36fbe0627d55f4a16763269202aaf9411998c6a578ec9c3978a


In [ ]:
def get_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = x_train.reshape(x_train.shape + (1,)).astype("float32")
    x_test = x_test.reshape(x_test.shape + (1,)).astype("float32")

    x_train /= 256.0
    x_test /= 256.0

    x_mean = np.mean(x_train, axis=0)

    x_train -= x_mean
    x_test -= x_mean

    nb_classes = np.max(y_train)+1
    y_train = to_categorical(y_train, nb_classes)
    y_test = to_categorical(y_test, nb_classes)

    # # for int 4
    # quantizer = quantized_bits(4, 1)
    # # for int 8
    # quantizer = quantized_bits(8, 1)
    # # for int 12
    # quantizer = quantized_bits(12, 1)
    # # for int 16
    # quantizer = quantized_bits(16, 1)
    # For int 32
    quantizer = quantized_bits(32, 1)


    x_train = quantizer(x_train).numpy()
    x_test = quantizer(x_test).numpy()

    return (x_train, y_train), (x_test, y_test)

(x_train, y_train), (x_test, y_test) = get_data()

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
# from keras import backend as K
# class MyActivation(keras.layers.Layer):
#     def __init__(self, **kwargs):
#         super(MyActivation, self).__init__(**kwargs)

#     def build(self, input_shape):
#         # Create a trainable weight variable for this layer.
#         self.a = self.add_weight(name='a',
#                                       shape=(1),
#                                       initializer ='ones',  # TODO: Choose your initializer
#                                       trainable=True)
#         self.c = self.add_weight(name='c',
#                                       shape=(1),
#                                       initializer='ones',  # TODO: Choose your initializer
#                                       trainable=True)
#         super(MyActivation, self).build(input_shape)

#     def call(self, x):
#         #return tf.matmul(inputs, self.w) + self.b
#         #quantizer = quantized_bits(9, 1)
#         return K.maximum(self.a*(x + self.c), K.minimum(self.a*(x - self.c) + self.c, x))
#         #return K.maximum(K.zeros_like(x), x)

#     def compute_output_shape(self, input_shape):
#         return input_shape

In [ ]:
# from keras import backend as K
# def swish(x, beta=10.0):
#     #Here return the c1*x + c2
#     global intervals, coeffArray
#     #idx = np.array(intervals).searchsorted(x.data())
#     idx = quantized_bits(9,1)(x)/0.03125
#     coeff = np.array(coeffArray)
#     return coeff[idx][0]*x + coeff[idx][1]*K.ones_like(x)
#     a = tf.gather(coeff, int(idx))
#     return x

#     #K.stop_gradient()

In [ ]:
# ### Here I'll try to use the method outlined in this link
# # https://stackoverflow.com/questions/54204393/piecewise-activation-function-in-tensorflow-and-broadcasting-math-operation

# def swish(x):
#   global intervals, coeffArray
#   coeff = np.array(coeffArray)
#   conditionArray = sum([tf.multiply(tf.cast(tf.math.logical_and(tf.math.less(x, 0.03125*(n+1)), tf.math.greater_equal(x, 0.03125*n)), tf.float32), coeff[n][0]*x + coeff[n][1]*K.ones_like(x)) for n in range(256)])

#   return conditionArray

Here below, uncomment the block according to whether int4, int8, int12 or int16 or fp32

In [ ]:
from qkeras import *
import qkeras


#for int4
# def CreateModel(shape, nb_classes, intBits):
#     x = x_in = Input(shape)
#     x = Flatten(name="flatten")(x)

#     x = QDense(256,
#         kernel_quantizer="quantized_bits(4, {} , alpha=1)".format(intBits),
#         bias_quantizer="quantized_bits(4, {} , alpha=1)".format(intBits),
#         name="dense")(x)

#     # Activation layer
#     x = Activation('relu')(x)

#     x = QDense(128,
#         kernel_quantizer="quantized_bits(4, {} , alpha=1)".format(intBits),
#         bias_quantizer="quantized_bits(4, {} , alpha=1)".format(intBits),
#         name="dense2")(x)

#     # Activation layer
#     x = Activation('relu')(x)

#     x = QDense(128,
#         kernel_quantizer="quantized_bits(4, {} , alpha=1)".format(intBits),
#         bias_quantizer="quantized_bits(4, {} , alpha=1)".format(intBits),
#         name="dense3")(x)

#     # Activation layer
#     x = Activation('relu')(x)

#     x = QDense(nb_classes,
#         kernel_quantizer="quantized_bits(4, {} , alpha=1)".format(intBits),
#         bias_quantizer="quantized_bits(4, {} , alpha=1)".format(intBits),
#         name="dense4")(x)

#     # Output layer with softmax activation
#     x = Activation("softmax", name="softmax")(x)

#     model = Model(inputs=x_in, outputs=x)
#     return model


#For 8 bits
# def CreateModel(shape, nb_classes, intBits):
#     x = x_in = Input(shape)
#     x = Flatten(name="flatten")(x)

#     x = QDense(256,
#         kernel_quantizer="quantized_bits(9, {} , alpha=1)".format(intBits),
#         bias_quantizer="quantized_bits(9, {} , alpha = 1)".format(intBits),
#         name="dense")(x)

#     #x = MyActivation()(x)
#     #x = Activation(swish)(x)
#     x = Activation('relu')(x)

#     x = QDense(128,
#         kernel_quantizer="quantized_bits(9, {} , alpha=1)".format(intBits),
#         bias_quantizer="quantized_bits(9, {} , alpha=1)".format(intBits),
#         name="dense2")(x)

#     #x = MyActivation()(x)
#     #x = Activation(swish)(x)
#     x = Activation('relu')(x)

#     x = QDense(128,
#         kernel_quantizer="quantized_bits(9, {} , alpha=1)".format(intBits),
#         bias_quantizer="quantized_bits(9, {} , alpha=1)".format(intBits),
#         name="dense3")(x)

#     #x = MyActivation()(x)
#     #x = Activation(swish)(x)
#     x = Activation('relu')(x)

#     x = QDense(nb_classes,
#         kernel_quantizer="quantized_bits(9, {} , alpha=1)".format(intBits),
#         bias_quantizer="quantized_bits(9, {} , alpha=1)".format(intBits),
#         name="dense4")(x)

#     x = Activation("softmax", name="softmax")(x)

#     model = Model(inputs=x_in, outputs=x)
#     return model


# for int 12
# def CreateModel(shape, nb_classes, intBits):
#     x = x_in = Input(shape)
#     x = Flatten(name="flatten")(x)

#     x = QDense(256,
#         kernel_quantizer="quantized_bits(12, {} , alpha=1)".format(intBits),
#         bias_quantizer="quantized_bits(12, {} , alpha=1)".format(intBits),
#         name="dense")(x)

#     # Activation layer
#     x = Activation('relu')(x)

#     x = QDense(128,
#         kernel_quantizer="quantized_bits(12, {} , alpha=1)".format(intBits),
#         bias_quantizer="quantized_bits(12, {} , alpha=1)".format(intBits),
#         name="dense2")(x)

#     # Activation layer
#     x = Activation('relu')(x)

#     x = QDense(128,
#         kernel_quantizer="quantized_bits(12, {} , alpha=1)".format(intBits),
#         bias_quantizer="quantized_bits(12, {} , alpha=1)".format(intBits),
#         name="dense3")(x)

#     # Activation layer
#     x = Activation('relu')(x)

#     x = QDense(nb_classes,
#         kernel_quantizer="quantized_bits(12, {} , alpha=1)".format(intBits),
#         bias_quantizer="quantized_bits(12, {} , alpha=1)".format(intBits),
#         name="dense4")(x)

#     # Output layer with softmax activation
#     x = Activation("softmax", name="softmax")(x)

#     model = Model(inputs=x_in, outputs=x)
#     return model


#For 16 bits
# def CreateModel(shape, nb_classes, intBits):
#     x = x_in = Input(shape)
#     x = Flatten(name="flatten")(x)

#     x = QDense(256,
#         kernel_quantizer="quantized_bits(16, {} , alpha=1)".format(intBits),
#         bias_quantizer="quantized_bits(16, {} , alpha = 1)".format(intBits),
#         name="dense")(x)

#     # Activation layer
#     #x = Activation(swish)(x)
#     x = Activation('relu')(x)

#     x = QDense(128,
#         kernel_quantizer="quantized_bits(16, {} , alpha=1)".format(intBits),
#         bias_quantizer="quantized_bits(16, {} , alpha=1)".format(intBits),
#         name="dense2")(x)

#     # Activation layer
#     #x = Activation(swish)(x)
#     x = Activation('relu')(x)

#     x = QDense(128,
#         kernel_quantizer="quantized_bits(16, {} , alpha=1)".format(intBits),
#         bias_quantizer="quantized_bits(16, {} , alpha=1)".format(intBits),
#         name="dense3")(x)

#     # Activation layer
#     #x = Activation(swish)(x)
#     x = Activation('relu')(x)

#     x = QDense(nb_classes,
#         kernel_quantizer="quantized_bits(16, {} , alpha=1)".format(intBits),
#         bias_quantizer="quantized_bits(16, {} , alpha=1)".format(intBits),
#         name="dense4")(x)

#     # Output layer with softmax activation
#     x = Activation("softmax", name="softmax")(x)

#     model = Model(inputs=x_in, outputs=x)
#     return model


# For 32 bits
def CreateModel(shape, nb_classes, intBits):
    x = x_in = Input(shape)
    x = Flatten(name="flatten")(x)

    x = QDense(256,
        kernel_quantizer="quantized_bits(32, {} , alpha=1)".format(intBits),
        bias_quantizer="quantized_bits(32, {} , alpha = 1)".format(intBits),
        name="dense")(x)

    # Activation layer
    x = Activation('relu')(x)

    x = QDense(128,
        kernel_quantizer="quantized_bits(32, {} , alpha=1)".format(intBits),
        bias_quantizer="quantized_bits(32, {} , alpha=1)".format(intBits),
        name="dense2")(x)

    # Activation layer
    x = Activation('relu')(x)

    x = QDense(128,
        kernel_quantizer="quantized_bits(32, {} , alpha=1)".format(intBits),
        bias_quantizer="quantized_bits(32, {} , alpha=1)".format(intBits),
        name="dense3")(x)

    # Activation layer
    x = Activation('relu')(x)

    x = QDense(nb_classes,
        kernel_quantizer="quantized_bits(32, {} , alpha=1)".format(intBits),
        bias_quantizer="quantized_bits(32, {} , alpha=1)".format(intBits),
        name="dense4")(x)

    # Output layer with softmax activation
    x = Activation("softmax", name="softmax")(x)

    model = Model(inputs=x_in, outputs=x)
    return model

In [ ]:
from tensorflow.keras.optimizers import Adam
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=2, restore_best_weights=True)

model = CreateModel(x_train.shape[1:], y_train.shape[-1], 1)
model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(0.0005),
    #optimizer='sgd',
    metrics=["accuracy"],)
history = model.fit(x_train, y_train, epochs=5, batch_size=128, validation_data=(x_test[:5000], y_test[:5000]), verbose=False, callbacks=[callback])

In [ ]:
model.evaluate(x_test[5000:], y_test[5000:])

157/157 [==============================] - 1s 5ms/step - loss: 0.0457 - accuracy: 0.9850


[0.04570109024643898, 0.9850000143051147]

Finally we get the test accuracy here !